<h4> MA Crossings </h4>
<h5> Creating a simple 200 to 50 MA crossing simulation on jupyter. </h5>
<h6> Will use previous bollinger band program for the simulation. </h6>

# Configurations

In [9]:
DATASET = "Bitstamp_ETHUSD_1h.csv"
DATASET_HEADER = "unix,open,high,low,close,Volume USD"
TX_SIZE = 1

# Libraries

In [10]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Definitions

In [11]:
def import_csv():
    '''Imports csv to pandas dataframe.'''
    print(f"Importing {DATASET} as df...")
    df = pd.read_csv(DATASET) # csv to df
    df = df.sort_values(by=["date"], ascending=True) # Sort data by ascending date
    df["date"] = pd.to_datetime(df["date"], unit = "s") # Converting epoch to datetime
    print("Import successful.")
    return df

def get_sma(closing_price, SMA_PERIOD):
    ''' Gets simple moving average with SMA_PERIOD as indicated by config.py.'''
    return closing_price.rolling(SMA_PERIOD).mean()

def check_param(df):
    ''' Go through each data in the dataframe to look for buy and sell signals.'''
    df.dropna(inplace=True) # Drop rows with NaNs
    df.iloc[::-1] # Reverse index
    prev_sig, sig, logbook = False,False, [] #F->T BUY, T->F SELL
    # Problem: It buys the first signal as long as SMA200 is above SMA50
    for i in df.index:
        #print(f"Processing {(len(df) - 1)-i} of {(len(df) - 1)}")
        prev_sig = sig
        sig = False if(df['sma_200'][i] > df['sma_50'][i]) else True
        if prev_sig != sig: transact(i, sig, logbook)
    return logbook


def transact(i, sig, logbook):
    '''Buys or sells, depending on the signal'''
    logs = {}
    logs['datetime'] = df['date'][i]
    logs['price'] = df['close'][i]
    logs['size'] = TX_SIZE
    if sig: #Buy
        logs['type'] = "BUY"
        logbook.append(logs.copy())
        logs.clear()
    else: #Sell
        logs['type'] = "SELL"
        logbook.append(logs.copy())
        logs.clear()

def export_csv(tx):
    ''' Export 2 csv files - one containing sold positions and one with open positions.'''
    with open('output.csv', 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["datetime","price", "size", "type"])
        writer.writeheader()
        writer.writerows(tx)


# Main

In [12]:
if __name__ == '__main__':
    df = import_csv()
    df['sma_200'] = get_sma(df['close'],200)
    df['sma_50'] = get_sma(df['close'],50)
    logbook = check_param(df)

    print("Calculating profits.")
    profit, profit_book = 0, []
    for i in range(1,len(logbook)):
        if i%2 == 1:
            x = logbook[i]['price'] - logbook[i-1]['price']
            profit += x
            profit_book.append(x)
    export_csv(logbook)
    with open("profit_book.txt","w") as f:
        for i in profit_book:
            f.write(str(i)+"\n")
        f.write(f"Total profit is {profit}")
    print("Done.")
    logs = pd.read_csv("output.csv")
    logs['datetime'] = pd.to_datetime(logs['datetime'])
    # PLOTTING
    for year in [2018,2019,2020,2021,2022]:
        data = df[df['date'].dt.year == year] # Filter by year
        logs_data = logs[logs['datetime'].dt.year == year] # Filter buys and sells by year
        buys = logs_data[logs_data['type'] == "BUY"] # Filter by 'type' w/ string BUY
        sells = logs_data[logs_data['type'] == "SELL"] # Filter by 'type' w/ string SELL
        fig = plt.figure(figsize=(60,16)) # 60 x 16 in plot
        ax = fig.add_subplot(111)# just 1 plot (x,y,z)
        ax.xaxis.set_major_locator(mdates.MonthLocator()) # set X axis ticks to be a month apart
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b')) # set X axis ticks format
        plt.yticks(np.arange(0, max(data['close']),max(data['close'])/20)) # Set Y axis ticks to be evenly spaced out in 20 divisions
        plt.title(f"ETHUSD YEAR {year}")
        plt.plot(data['date'],data['close']) # plotting of date,price
        plt.plot(data['date'],data['sma_200'],label="ETH 200 SMA",color='orange', zorder=0) #plotting of sma, zorder = layer number
        plt.plot(data['date'],data['sma_50'],label="ETH 50 SMA",color='gold', zorder=10)
        plt.scatter(buys['datetime'],buys['price'],label="Buys",color='green', zorder=20, marker="^", s=100) #plotting of buy signals
        for i,j in zip(buys['datetime'],buys['price']): # zip creates a list of lists from 2 lists. zip([a,b],[1,2]) results in ([a,1],[b,2])
            plt.text(i, j, j) # adding text
        plt.scatter(sells['datetime'],sells['price'],label="Sells",color='red', zorder=30, marker="v", s=100) #plotting of sell signals
        for i,j in zip(sells['datetime'],sells['price']):
            plt.text(i, j, j)
        plt.legend(loc='upper right') # Set legend location
        plt.show()# Show the plot

Importing btc.csv as df...


KeyError: 'date'